### Initialization

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import csv
import gc
import gzip
import json
from lxml import html
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import requests
import string
from string import ascii_letters
import sys
from tqdm import tqdm

# Dataset USCD Book Graph from GoodReads

In [ ]:
def load_data(file_name, head = 999999):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
            
            if (head is not None) and (count > head):
                break
    return data

In [ ]:
DIR = '/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/books/'

Preprocessing mainly needed for **MongoDB**

## Books

In [ ]:
book_dict = {
      'isbn' : '',
      'title' : '',
      'language' : '',
      'category' : '',
      'bookId' : 0,
      'publisher' : '',
      'description' : '',
      'numPages' : 0,
      'imageURL' : '',
      'author' : '',
      'publicationYear' : 0,
      'reviews' : [
      ]
}

### Function Definition

In [ ]:
dict_doppioni = {}

In [ ]:
len(dict_doppioni)

219861

You can reuse the pre-computed dictionary_doppioni

In [ ]:
def create_json_book(file, category_array, filejson):
  books = load_data(os.path.join(DIR,file), head=99999)
  length = len(category_array)

  filename = 'books_' + filejson + '.json'

  # create the file
  f = open(filename, "x")
  f.close()

  with open(filename, 'a', encoding='utf-8') as f:
    f.write('[')
    i = 0
    for libro in books:
      book_json = book_dict
      
      # skip books with missing parameters
      if libro['isbn13'] == '' or libro['language_code'] == '' or libro['num_pages'] == '':
        continue

      if libro['isbn13'] in dict_doppioni:
        continue
      else:
        dict_doppioni[libro['isbn13']] = True

      if i != 0:
        f.write(',')

      book_json['title'] = libro['title']
      book_json['isbn'] = libro['isbn13']
      book_json['bookId'] = libro['book_id']
      if len(libro['authors']) != 0:
        book_json['author'] = mapping_id_author[libro['authors'][0]['author_id']]
      else:
        book_json['author'] = 'Unknown'

      book_json['language'] = libro['language_code']
      book_json['category'] = category_array[random.randint(0, length-1)]
      book_json['description'] =  libro['description']
      book_json['publisher'] = libro['publisher']
      book_json['numPages'] = int(libro['num_pages'])
      book_json['publicationYear'] = libro['publication_year']
      book_json['imageURL'] = libro['image_url']

      i = i + 1

      json.dump(book_json, f, ensure_ascii=False, indent=4)

    f.write(']')

  return 

In [ ]:
def return_list_book_and_titles(file):
  books = load_data(os.path.join(DIR,file), head=99999)

  list_books = []
  list_titles = []
  list_book_id = []
  list_img = []
  list_author = []

  for libro in books:
    if libro['isbn13'] == '' or libro['language_code'] == '' or libro['num_pages'] == '':
      continue

    if libro['isbn13'] in dict_doppioni:
      continue
    else:
      dict_doppioni[libro['isbn13']] = True
    
    list_books.append(libro['isbn13'])
    list_titles.append(libro['title'])
    list_book_id.append(libro['book_id'])
    list_img.append(libro['image_url'])

    if len(libro['authors']) != 0:
      list_author.append(mapping_id_author[libro['authors'][0]['author_id']])
    else:
      list_author.append('Unknown')

  return list_books, list_titles, list_book_id, list_img, list_author

In [ ]:
b1, t1, id1, im1, a1  = return_list_book_and_titles('goodreads_books_children.json.gz')
b2, t2, id2, im2, a2  = return_list_book_and_titles('goodreads_books_comics_graphic.json.gz')
b3, t3, id3, im3, a3  = return_list_book_and_titles('goodreads_books_fantasy_paranormal.json.gz')
b4, t4, id4, im4, a4  = return_list_book_and_titles('goodreads_books_history_biography.json.gz')
b5, t5, id5, im5, a5  = return_list_book_and_titles('goodreads_books_mystery_thriller_crime.json.gz')
b6, t6, id6, im6, a6  = return_list_book_and_titles('goodreads_books_poetry.json.gz')
b7, t7, id7, im7, a7 = return_list_book_and_titles('goodreads_books_romance.json.gz')
b8, t8, id8, im8, a8  = return_list_book_and_titles('goodreads_books_young_adult.json.gz')

In [ ]:
list_isbn = b1 + b2 + b3 + b4 + b5 + b6 + b7 + b8
list_titles  = t1 + t2 + t3 + t4 + t5 + t6 + t7 + t8
list_bookid = id1 + id2 + id3 + id4 + id5 + id6 + id7 + id8
list_img_url = im1 + im2 + im3 + im4 + im5 + im6 + im7 + im8
list_authors = a1 + a2 + a3 + a4 + a5 + a6 + a7 + a8 

Dictionaries useful later for Reviews and other stuff.

In [ ]:
dictionary_bookid_to_isbn = dict(zip(list_bookid,list_isbn))
dictionary_bookid_to_title = dict(zip(list_bookid,list_titles))

### Creation of JSON file

Create JSON file for books category.

In [ ]:
create_json_book('goodreads_books_children.json.gz',['Picture Book', 'Easy Reader'],'children')
create_json_book('goodreads_books_comics_graphic.json.gz',['Comics','Graphic Novels'],'comics')
create_json_book('goodreads_books_fantasy_paranormal.json.gz',['Fantasy','Paranormal'],'fantasy')
create_json_book('goodreads_books_history_biography.json.gz',['History','Biography'],'history')
create_json_book('goodreads_books_mystery_thriller_crime.json.gz',['Mystery','Thriller','Crime'],'crime')
create_json_book('goodreads_books_poetry.json.gz',['Love Poems', 'Anthology', 'Poetry'],'poetry')
create_json_book('goodreads_books_romance.json.gz',['Erotic Romance', 'Romantic Suspense', 'Romance'],'romance')
create_json_book('goodreads_books_young_adult.json.gz',['Adventures','Diary','Family and Relationship','Young Fiction'],'young')

* Children:(**28648**)
* Comics:(**38464**)
* Fantasy:(**37119**)
* History:(**29735**)
* Mistery:(**35265**)
* Poetry:(**10180**)
* Romance:(**27348**)
* Young:(**39807**)

For a total of **246.566** books.

## Authors

Dictionary that maps *author_id* into name of the **author**.

In [ ]:
mapping_id_author = {}
autori = load_data('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/goodreads_book_authors.json.gz')

In [ ]:
for aut in tqdm(autori):
  id = aut['author_id']
  name = aut['name']
  mapping_id_author[id] = name

100%|██████████| 829529/829529 [00:01<00:00, 460153.35it/s]


## Reviews

In [ ]:
DIR = '/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/reviews/'

In [ ]:
def return_df_review_category(file):
  reviews = load_data(os.path.join(DIR,file), head=75000)

  list_users = []
  list_isbn = []
  list_titles = []
  list_review_text = []
  list_ratings = []
  list_reviewId = []

  for review in tqdm(reviews):

    book_id = review['book_id']
    print(review)
    # controllo necessario poiché alcuni libri non li abbiamo presi in considerazione
    # e qui ci sono invece tutte le recensioni 
    if book_id in dictionary_bookid_to_isbn:
      print("a")
      pass
    else:
      continue

    # alcune review non hanno il testo
    if 'review_text' in review:
      list_review_text.append(review['review_text'])
    else:
      list_review_text.append('')

    username = list_usernames[random.randint(0,len(list_usernames)-1)]
    list_users.append(username)
    isbn = dictionary_bookid_to_isbn[book_id]
    list_isbn.append(isbn)
    list_titles.append(dictionary_bookid_to_title[book_id])
    
    list_ratings.append(review['rating'])
    review_id = username + ":" + isbn
    list_reviewId.append(review_id)

  reviews_df = pd.DataFrame(list(zip(list_users, list_isbn, list_titles, list_review_text, list_ratings, list_reviewId)), columns =['Username','ISBN','Title','Review_Text','Rating','reviewID'])

  return reviews_df

In [ ]:
children_df = return_df_review_category('goodreads_reviews_children.json.gz')

100%|██████████| 75001/75001 [00:00<00:00, 1091932.42it/s]


In [ ]:
children_df = return_df_review_category('goodreads_reviews_children.json.gz')
children_df.to_csv("children_df_review.csv")
print("children children  children\n" + str(children_df.info()))

comics_df = return_df_review_category('goodreads_reviews_comics_graphic.json.gz')
comics_df.to_csv("comics_df_review.csv")
print("comics comics  comics\n" + str(comics_df.info()))

poetry_df = return_df_review_category('goodreads_reviews_poetry.json.gz')
poetry_df.to_csv("poetry_df_review.csv")
print("poetry poetry  poetry\n" + str(poetry_df.info()))

romance_df = return_df_review_category('goodreads_reviews_romance.json.gz')
romance_df.to_csv("romance_df_review.csv")
print("romance romance  romance\n" + str(romance_df.info()))

fantasy_df = return_df_review_category('goodreads_reviews_fantasy_paranormal.json.gz')
fantasy_df.to_csv("fantasy_df_review.csv")
print("fantasy fantasy  fantasy\n" + str(fantasy_df.info()))

young_df = return_df_review_category('goodreads_reviews_young_adult.json.gz')
young_df.to_csv("young_df_review.csv")
print("young young  young\n" + str(young_df.info()))

mistery_df = return_df_review_category('goodreads_reviews_mystery_thriller_crime.json.gz')
mistery_df.to_csv("mistery_df_review.csv")
print("mistery mistery  mistery\n" + str(mistery_df.info()))

history_df = return_df_review_category('goodreads_reviews_history_biography.json.gz')
history_df.to_csv("history_df_review.csv")
print("history history  history\n" + str(history_df.info()))

Number of **Reviews** taken by *Category*:

* Children:(**32071**)
* Comics:(**50446**)
* Fantasy:(**29629**)
* History:(**15333**)
* Mistery:(**22195**)
* Poetry:(**34821**)
* Romance:(**13414** )
* Young:(**46604**)

For a total number of **253.977** Reviews.


## Users

In [ ]:
names = []
with open('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/RandomNames.csv', newline='') as f:
    reader = csv.reader(f)
    data = [tuple(row) for row in reader]
    names.append(data)


In [ ]:
list_names = []
for row in names[0]:
  list_names.append(row[0])

In [ ]:
len(list_names)

200000

In [ ]:
list_names_deduplicate = list(dict.fromkeys(list_names))

In [ ]:
list_usernames = []
for name in list_names_deduplicate:
  list_string = name.split()
  if list_string[0] == 'Mrs.' or list_string[0] == 'Mr.' or list_string[0] == 'Miss.' or list_string[0] == 'Ms.' or list_string[0] == 'Dr.':
    string = list_string[1] + '_' + list_string[2]
  else:
    string = list_string[0] + '_' + list_string[1]
  
  list_usernames.append(string)

In [ ]:
len(list_usernames)

119961

In [ ]:
DEDUPLICATO = list(set(list_usernames))

In [ ]:
list_usernames = DEDUPLICATO

In [ ]:
len(list_usernames)

115449

It is useful to retrieve the original name and surname

In [ ]:
list_names = []
list_surnames = []
for username in list_usernames:
  string = username.split('_')
  list_names.append(string[0])
  list_surnames.append(string[1])


Per la **mail** fai name.surname@gmail.com.

Per la **password** inventa una combinazione alfanumerica di 8 cifre.

Per la lista di **favourite** e le **readingList** direi di creare una lista di ISBN presenti nella nostra applicazione, poi sceglierne il numero che vogliamo a caso nella lista.

In [ ]:
user_dict = {
             'username' : '', 
             'name' : '', 
             'surname' : '', 
             'email' : '',
             'numReviews' : 0,
             'password' : '',
             'isAdministrator' : False,
             'favourite' : [ ],
             'reviews' : [ ],
             'readingList' : [ ]
}

**ReviewID** è l'id del **reviewer** quindi dell'*utente* due punti l'isbn del libro, considerando che un utente può fare una sola review di un libro.

Qui creato lo **users.json**

In [ ]:
import string

In [ ]:
with open('users.json', 'a', encoding='utf-8') as f:

  # note that the file must exists yet to function
  i = 0
  f.write('[')
  for name in list_usernames:
    if i != 0:
      f.write(',')

    data = user_dict
    list_string = name.split('_')
    data['username'] = list_string[0] + '_' + list_string[1]
    data['name'] = list_string[0]
    data['surname'] = list_string[1]
    data['email'] = list_string[1] + '.' + list_string[0] + '@gmail.com'
    data['password'] = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
    json.dump(data, f, ensure_ascii=False, indent=4)
    i = i+1
    
  f.write(']')


print(i)

115449


## Create Favourite Dataframe

In [ ]:
fav_isbn = []
fav_username = []
fav_title = []
fav_imageURL = []
fav_author = []


map_isbn_user = {}

for i in tqdm(range(0,400000)):
  index = random.randint(0,len(list_isbn)-1)

  # se l'isbn è nel dictionary
  if list_isbn[index] in map_isbn_user:
    list_user_per_isbn = map_isbn_user[list_isbn[index]]
  else: # se non lo è creo la sua key e metto array vuoto
    map_isbn_user[list_isbn[index]] = []
    list_user_per_isbn = []

  user = list_usernames[random.randint(0,len(list_usernames)-1)]

  if user in list_user_per_isbn:
    continue
  else:
    list_user_per_isbn.append(user)
    map_isbn_user[list_isbn[index]] = list_user_per_isbn

  fav_isbn.append(list_isbn[index])
  fav_username.append(user)
  fav_title.append(list_titles[index])
  fav_imageURL.append(list_img_url[index])
  fav_author.append(list_authors[index])


favourite_df = pd.DataFrame(list(zip(fav_isbn, fav_username, fav_title, fav_imageURL, fav_author)), columns =['ISBN','username','title','imageURL','author'])
  
  
 

100%|██████████| 400000/400000 [00:08<00:00, 47554.07it/s] 


In [ ]:
favourite_df

,ISBN,username,title,imageURL,author
0,9781469984018,Terrence_Roth,The Boy Who Sneaks in My Bedroom Window,https://images.gr-assets.com/books/1398682775m...,Kirsty Moseley
1,9780988028623,Barbara_Kennedy,Sleepy Bunny - A Nap Time Tail,https://images.gr-assets.com/books/1362159817m...,Gregory Isherwood
2,9780316099691,Calvin_Hall,The Matchmaker,https://images.gr-assets.com/books/1415581022m...,Elin Hilderbrand
3,9780374173616,Rachel_Gilbert,Metaphysical Dog,https://images.gr-assets.com/books/1376835260m...,Frank Bidart
4,9781480568372,Stephen_Greene,Love Letters to the Dead,https://images.gr-assets.com/books/1395614083m...,Ava Dellaira
...,...,...,...,...,...
399992,9780987155139,Anna_Davis,"Cyanide and Poppies (Eleanor Jones, #2)",https://images.gr-assets.com/books/1354864121m...,Carolyn Morwood
399993,9789792257755,Matthew_Potts,Ajdar,https://images.gr-assets.com/books/1275464868m...,Marjane Satrapi
399994,9785410007511,Brandon_Hardy,Vilkaču mantiniece,https://images.gr-assets.com/books/1427981774m...,Ilona Leimane
399995,9786076212912,Barbara_Stevens,Tanto que contarte,https://images.gr-assets.com/books/1454980911m...,John Marsden


In [ ]:
favourite_df.to_csv("favourite_df.csv")

## Create ReadingList Dataframe

In [ ]:
!pip install random-word

from random_word import RandomWords
r = RandomWords()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 12.2 MB/s 


In [ ]:
rl_username = []

# queste sono liste nel df
rl_isbns = []
rl_titles = []
rl_imageURLs = []
rl_authors = []

rl_name = []

# 30k readingList
for i in tqdm(range(0,5000)):
  user = list_usernames[random.randint(0,len(list_usernames)-1)]
  rl_username.append(user)

  name_reading_list = r.get_random_word() + " " + r.get_random_word()
  rl_name.append(name_reading_list)

  isbns = []
  titles = []
  imageURLs = []
  authors = []

  # a number between 0 and 5 books in one ReadingList
  for j in range(0,random.randint(1,5)):
    index = random.randint(0,len(list_isbn)-1)

    # same book not allowed in the reading list
    if list_isbn[index] in isbns:
      continue

    isbns.append(list_isbn[index])
    titles.append(list_titles[index])
    imageURLs.append(list_img_url[index])
    authors.append(list_authors[index])


  rl_isbns.append(isbns)
  rl_titles.append(titles)
  rl_imageURLs.append(imageURLs)
  rl_authors.append(authors)

rl_df = pd.DataFrame(list(zip(rl_isbns, rl_username, rl_titles, rl_imageURLs, rl_authors, rl_name)), columns =['ISBN','username','title','imageURL','author','name'])
  
  
 

100%|██████████| 5000/5000 [52:01<00:00,  1.60it/s]


In [ ]:
rl_df

,name
0,Douglas_Meyer:florinda sorediferous
1,Andrea_Macdonald:kinaesthesias canonizant
2,Laura_Booker:ibidinae telefilms
3,Robin_Hoover:silicious quinoidation
4,Danielle_Schwartz:alcestis introits
...,...
4995,Kristin_Jenkins:tropeolin chamaerrhine
4996,Scott_Gentry:fittiest prand
4997,George_Gray:micracoustic transe
4998,Steve_Ross:externalistic sensitizing


In [ ]:
# mongodb -> da affiancare al df rl_df
list_numlikes = []

# neo4j
list_names = []
list_likers = []


for index, row in rl_df.iterrows():
  num_likes = random.randint(1,20)
  list_numlikes.append(num_likes)
  for i in range(0,num_likes):
    list_likers.append(list_usernames[random.randint(0,len(list_usernames)-1)])
    rlname = row['name']
    list_names.append(rlname)
    

In [ ]:
rl_df['numLikes'] = list_numlikes

In [ ]:
rl_df.to_csv("reading_list_df.csv")

## Neo4J preprocessing data

We must generate 4 entities and 6 relations:

*   **User**
*   **ReadingList**
*   **Book**
*   **Category**
*   User-User -> *Follow*
*   User-ReadingList -> *Likes*
*   ReadingList-Book -> *Has*
*   Book-Category -> *Belongs_to*
*   User-Category -> *Is_interested_to*
*   User-Book -> *Favorites*





**User**

In [ ]:
user_df = pd.DataFrame(list(zip(list_usernames,list_names,list_surnames)), columns =['username','name','surname'])
user_df.to_csv("user_entity.csv", index=False)

**Category**

In [ ]:
CATEGORIES = ['Picture Book', 'Easy Reader', 'Comics','Graphic Novels', 'Fantasy','Paranormal',
              'History','Biography', 'Mystery','Thriller','Crime', 'Love Poems',
              'Anthology', 'Poetry', 'Erotic Romance', 'Romantic Suspense', 'Romance',
              'Adventures','Diary','Family and Relationship','Young Fiction']

In [ ]:
categories_df = pd.DataFrame(CATEGORIES, columns =['category'])
categories_df.to_csv("category_entity.csv", index=False)

**Book**

In [ ]:
book_df = pd.DataFrame(list(zip(list_isbn,list_titles,list_authors,list_img_url)), columns =['isbn','title','author','image'])
book_df.to_csv(
    "book.csv",
    index=False)

**Reading List**

In [ ]:
rl_df = pd.read_csv('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/reading_list_df.csv')

In [ ]:
list_rl = []

for index, row in rl_df.iterrows():
  rl = row['username'] + ":" + row['name']
  list_rl.append(rl)

In [ ]:
rl_df = pd.DataFrame(list_rl, columns =['name'])
rl_df.to_csv("readinglist.csv",index=False)

*Follow*

In [ ]:
list_src = []
list_dst = []
for i in range(0,500000):
  src = list_usernames[random.randint(0,len(list_usernames)-1)]
  dst = list_usernames[random.randint(0,len(list_usernames)-1)]
  if src != dst:
    list_src.append(src)
    list_dst.append(dst)

In [ ]:
follow_df = pd.DataFrame(list(zip(list_src,list_dst)), columns =['src','dst'])
follow_df.to_csv("follow.csv",index=False)

*Belongs_to*

In [ ]:
df_children = pd.read_json('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/books_json/books_children.json')
df_comics = pd.read_json('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/books_json/books_comics.json')
df_crime = pd.read_json('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/books_json/books_crime.json')
df_fantasy = pd.read_json('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/books_json/books_fantasy.json')
df_history = pd.read_json('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/books_json/books_history.json')
df_poetry = pd.read_json('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/books_json/books_poetry.json')
df_romance = pd.read_json('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/books_json/books_romance.json')
df_young = pd.read_json('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/books_json/books_young.json')

In [ ]:
df_children = df_children[['isbn', 'category']]
df_comics = df_comics[['isbn', 'category']]
df_crime = df_crime[['isbn', 'category']]
df_fantasy = df_fantasy[['isbn', 'category']]
df_history = df_history[['isbn', 'category']]
df_poetry = df_poetry[['isbn', 'category']]
df_romance = df_romance[['isbn', 'category']]
df_young = df_young[['isbn', 'category']]

In [ ]:
frames = [df_children, df_comics, df_crime, df_fantasy, df_history, df_poetry, df_romance, df_young]
df_complete = pd.concat(frames)
df_complete.to_csv("belongs_books.csv",index=False)

*Is_interested_to*

In [ ]:
length_cat = len(CATEGORIES)
list_user = []
list_interests = []
for i in range(0,len(list_usernames)):
  for j in range(0,random.randint(1,4)):
    list_user.append(list_usernames[i])
    list_interests.append(CATEGORIES[random.randint(0,length_cat-1)])


In [ ]:
isinterested_df = pd.DataFrame(list(zip(list_user,list_interests)), columns =['user','interest'])
isinterested_df.to_csv("isinterested.csv",index=False)

*Favorites*

In [ ]:
df_favorites = pd.read_csv('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/favourite_df.csv')

In [ ]:
df_favorites = df_favorites[['ISBN', 'username']]
df_favorites.to_csv("favorites.csv",index=False)

*Has*

In [ ]:
df_rlhas = pd.read_csv('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/reading_list_df.csv')

In [ ]:
def Convert(string):
  string = string[1:-1]
  li = list(string.split(", "))
  return li

In [ ]:
list_rl = []
list_books = []

for index, row in df_rlhas.iterrows():
  rl = row['username'] + ":" + row['name']
  list_tmp = Convert(row['ISBN'])
  for book in list_tmp:
    list_rl.append(rl)
    list_books.append(book[1:-1])

In [ ]:
readinglist_df = pd.DataFrame(list(zip(list_rl,list_books)), columns =['name','book'])
readinglist_df.to_csv("rl_has.csv",index=False)

*Likes*

In [ ]:
reading_list_df = pd.read_csv('/content/gdrive/Shareddrives/LSMSD-BookSocialNetwork/reading_list_df.csv')

In [ ]:
list_likers = []
list_names = []

for index, row in reading_list_df.iterrows():
  rlname = row['username'] + ":" + row['name']
  numlikes = row['numLikes']
  for i in range(0,numlikes):
    list_likers.append(list_usernames[random.randint(0,len(list_usernames)-1)])
    list_names.append(rlname)


In [ ]:
likes_df = pd.DataFrame(list(zip(list_likers,list_names)), columns =['user','readinglist'])

In [ ]:
likes_df.to_csv("likes.csv",index=False)